# Seq2Seq with torch and torchtext
https://github.com/bentrevett/pytorch-seq2seq   
```English to German```

## Preparing Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchdata.datapipes.iter import IterDataPipe
from torchtext.datasets import Multi30k

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [4]:
def tokenize_de(text):
    '''
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    '''
    return [tok.text for tok in spacy_de.tokenizer(text)]
def tokenize_en(text):
    '''
    Tokenizes English text from a string into a list of strings(tokens)
    '''
    return [tok.text for tok in spacy_en.tokenizer(text)]


In [5]:
train_data, valid_data, test_data = Multi30k()

/Users/teddy/miniforge3/envs/Data/lib/python3.10/site-packages/torch/utils/data/datapipes/utils/common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(
/Users/teddy/miniforge3/envs/Data/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "


In [6]:
from torchtext.vocab import build_vocab_from_iterator
from itertools import tee, zip_longest
from functools import reduce

def yield_corpus(corpus):
    for src, trg in corpus:
        srcTokens = tokenize_de(src)
        trgTokens = tokenize_en(trg)
        yield srcTokens, trgTokens
    return

# Reverse Order
def yield_src(corpusIter):
    for src, _ in corpusIter:
        src.insert(0, '<sos>')
        src.append('<eos>')
        yield src
    return
        

def yield_trg(corpusIter):
    for _, trg in corpusIter:
        trg.insert(0, '<sos>')
        trg.append('<eos>')
        yield trg
    return

srcCorpusIter, trgCorpusIter = tee(yield_corpus(train_data))
srcIter, srcIterForVocab = tee(yield_src(srcCorpusIter))
trgIter, trgIterForVocab = tee(yield_trg(trgCorpusIter))

srcPadSize = reduce(lambda x, y: x if x > len(y) else len(y), srcIter, -1) + 2
trgPadSize = reduce(lambda x, y: x if x > len(y) else len(y), trgIter, -1) + 2

srcVocab = build_vocab_from_iterator(srcIterForVocab, min_freq = 2, special_first=True, specials=['<unk>', '<sos>', '<eos>', '<pad>'])
srcVocab.set_default_index(srcVocab['<unk>'])
trgVocab = build_vocab_from_iterator(trgIterForVocab, min_freq = 2, special_first=True, specials=['<unk>', '<sos>', '<eos>', '<pad>'])
trgVocab.set_default_index(trgVocab['<unk>'])

/Users/teddy/miniforge3/envs/Data/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/combining.py:180: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [7]:
print(srcPadSize)

48


In [8]:
from torchdata.datapipes.map import SequenceWrapper, Mapper
from functools import reduce
from copy import deepcopy

def build_src(src):
    src = deepcopy(src)
    src.insert(0, '<eos>')
    src.append('<sos>')
    return src

def build_trg(trg):
    trg = deepcopy(trg)
    trg.insert(0, '<sos>')
    trg.append('<eos>')
    return trg

def do_integer_encoding(pair):
    src, trg = pair
    encodedSrc = srcVocab.forward(src)
    encodedTrg = trgVocab.forward(trg)
    paddedSrc = encodedSrc + [srcVocab['<pad>'] for _ in range(srcPadSize - len(encodedSrc))]
    paddedTrg = encodedTrg + [trgVocab['<pad>'] for _ in range(trgPadSize - len(encodedTrg))]
    return torch.LongTensor(paddedSrc).to(device), torch.LongTensor(paddedTrg).to(device)

def build_corpus(pair):
    src, trg = deepcopy(pair)
    src = build_src(tokenize_de(src))
    trg = build_trg(tokenize_en(trg))
    return src, trg

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128

trainDP = SequenceWrapper(list(train_data)).map(build_corpus).map(do_integer_encoding)
validDP = SequenceWrapper(list(valid_data)).map(build_corpus).map(do_integer_encoding)
testDP = SequenceWrapper(list(test_data)).map(build_corpus).map(do_integer_encoding)



trainLoader = DataLoader(trainDP, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, )
validLoader = DataLoader(validDP, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, )
testLoader = DataLoader(testDP, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, )

## Build Seq2Seq Model

### Encoder
- 2-Layers
- LSTM returns ```outputs```, ```hidden```, and ```cell```
- To make context vector, only need ```hidden``` and ```cell```

In [9]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim, )
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src):
        # src: [src len, batch size]
        embedded = self.dropout(self.embedding(src)) # [src len, batch size, emb_dim]
        output, (hidden, cell) = self.lstm(embedded)
        # output = last hidden states, [src len, batch size, hid dim * n directions]
        # hidden = final hidden states, [n layers * n directions, batch size, hid dim]
        # cell = final cell states, [n layers * n directions, batch size, hid dim]
        
        return hidden, cell

### Context Vector
- At each layer, context vector is created by ```cell``` and ```hidden```
- If the number of layers at encoder and decoder are different, We make decision like passing same context vector? etc.

### Decoder
- Teacher force is used probabilitly
- Loop decoder until entering ```<eos>```
- Decoder poop starts at ```1```, not 0. So, 
```
trg = [<sos>, y1, y2, y3, <eos>]
pred = [0, p1, p2, p3, <eos>]
```
- Weh ncalculate the loss, cur off the first element of each tensor to get
```
trg = [y1, y2, y3, <eos>]
pred = [p1, p2, p3, <eos>]
```

In [10]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim ,emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout, batch_first = True)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, input_, hidden, cell):
        #input = [batchsize, token_size] <- a token [128]
        #hidden, cell <- passed by encoder first, [2, 128, 256]
        
        #Decoder's hidden, cell : [n layers * n directions, batch size, hid dim]
        input_ = input_.unsqueeze(1) #Make token a seq, [batch_size, 1]
        embedded = self.dropout(self.embedding(input_)) # [batch_size, 1, emb dim]
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell)) # With hidden and cell, make context
        
        prediction = self.fc_out(output.squeeze(0)) # [batch size, output dim]
        return prediction, hidden, cell # Re-passed by parameters

In [11]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimentions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layeres!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        #src: [batch size, src len]
        #trg: [batch size, trg len]
        
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        #Store decoder outputs
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[:, 0] #first input to the decoder is the <sos> tokens
        
        for t in range(1, trg_len): # output[0:] is zeros!!!!
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:,t,:] = output.squeeze()
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(0) # Find best prediction at each sample, [trg_len]
            input = trg[:, t] if teacher_force else top1
            return outputs

## Training

In [12]:
INPUT_DIM = len(srcVocab)
OUTPUT_DIM = len(trgVocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [13]:
len(srcVocab)

8014

### Weight normalization

In [14]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, +0.08)
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8014, 256)
    (lstm): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(6191, 256)
    (lstm): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=6191, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

### Count the parameters

In [15]:
def count_parameters(mode):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,168,879 trainable parameters


In [16]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = trgVocab['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [17]:
def train(model, iterator, optimizer, criterion, clip):
    model.train() # Set train mode <--> model.eval() : Set prediction mode with torch.no_grad() : Don't calcuate gradients
    
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src, trg = batch
        
        optimizer.zero_grad()
        output = model(src, trg)
        
        output_dim = output.shape[-1] # [trg len, batch size, output dim]
        output = output[1:].view(-1, output_dim) # Ignore first
        trg = trg[1:].view(-1) # Ignore first
        
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [18]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src, trg = batch
            output = model(src, trg, 0) # turn off teacher forcing
            
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [19]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins*60))
    return elapsed_mins, elapsed_secs

In [20]:
import time

N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(model, trainLoader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, validLoader, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss: .3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} | Val. PPL: {math.exp(valid_loss):7.3f}')

NameError: name 'epoch_secs' is not defined

In [21]:
model.load_state_dict(torch.load('tut1-model.pt'))
test_loss = evaluate(model, testLoader, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 8.281 | Test PPL: 3950.037 |
